In [1]:
%load_ext autoreload
%autoreload 2

# Experiments with EASE model

In [2]:
from pathlib import Path

import scipy.sparse
import numpy as np

import src.io as io
import src.evaluation as evaluation
import src.evaluation.splits
import src.evaluation.base

from src.algorithm.baseline.ease import EASE

## Datasets

In [3]:
## Uncomment corresponding directory.
## RETARGET True for Frappe, False for the others

DATA_DIR, RETARGET = Path('../../../data/CARS/Mobile_Frappe/'), True
# DATA_DIR, RETARGET = Path('../../../data/CARS/Food_com/'), False
# DATA_DIR, RETARGET = Path('../../../data/CARS/TripAdvisor/'), False

In [4]:
# Shouldn't need to change this info
INTERACTIONS = DATA_DIR / 'interactions.csv'

SEED = 123456
SEED2 = 78910

ITEM_ID = 'item'
USER_ID = 'user'

## Define algorithm and hyperparameter ranges

In [5]:
ALG = EASE

L2 = [10**p for p in range(1, 5)]
# L2 = [0.1 * (i + 1) for i in range(20)]

HYPERPARAMS = {'l2': L2}
HYPERPARAMS

{'l2': [10, 100, 1000, 10000]}

## Parse data

In [6]:
data = io.parse_interactions_with_context(INTERACTIONS, item_id=ITEM_ID, user_id=USER_ID)
data.df

,userId,itemId,daytime,weather,weekday
0,0,0,1,1,1
1,1,1,2,2,2
2,2,2,3,2,3
3,3,3,4,0,4
4,4,4,5,1,4
...,...,...,...,...,...
95997,109,0,3,1,1
95998,37,16,4,2,1
95999,180,33,2,2,1
96000,445,751,3,1,1


## Make train/val split for hyperparam tuning

In [7]:
Xtrain, Xval_in, Xval_out = evaluation.splits.leave_one_out_split_non_context(data, seed=SEED)

## Perform gridsearch on validation set

In [8]:
%%time
score, best_hyperparams = evaluation.base.gridsearch(ALG, Xtrain, Xval_in, Xval_out, HYPERPARAMS, retarget=RETARGET)
f"Best score of {score} achieved with {best_hyperparams}."

  0%|          | 0/4 [00:00<?, ?it/s]

Training model EASE with hyperparameters {'l2': 10}
Evaluating with 816 users
MRR@5 0.165
MRR@20 0.195
Average Recall@5 0.314
Average Recall@20 0.603
Training model EASE with hyperparameters {'l2': 100}
Evaluating with 816 users
MRR@5 0.104
MRR@20 0.133
Average Recall@5 0.207
Average Recall@20 0.499
Training model EASE with hyperparameters {'l2': 1000}
Evaluating with 816 users
MRR@5 0.086
MRR@20 0.108
Average Recall@5 0.18
Average Recall@20 0.388
Training model EASE with hyperparameters {'l2': 10000}
Evaluating with 816 users
MRR@5 0.083
MRR@20 0.103
Average Recall@5 0.175
Average Recall@20 0.363
CPU times: user 1min 40s, sys: 23 s, total: 2min 3s
Wall time: 9.82 s


"Best score of 0.19510039339547286 achieved with {'l2': 10}."

## Evaluate model with optimal hyperparams with KFolds Cross validation

In [9]:
%%time
alg = ALG(**best_hyperparams)
results = evaluation.base.kFoldsEval(alg, data, nr_folds=5, seed=SEED2, retarget=RETARGET)

Evaluating with 816 users
Evaluating with 816 users
Evaluating with 816 users
Evaluating with 816 users
Evaluating with 816 users
MRR@5 0.146 (0.004)
MRR@20 0.178 (0.005)
Average Recall@5 0.291 (0.005)
Average Recall@20 0.592 (0.014)
CPU times: user 2min 7s, sys: 28.2 s, total: 2min 35s
Wall time: 12.5 s
